Note. This is a summary of many failed experiments and not the actual notebook

In [ ]:
from pathlib import Path
from collections import Counter
from types import ModuleType
from typing import Optional
from IPython.display import display, HTML

import pyrosetta
import pyrosetta_help as ph
prc: ModuleType = pyrosetta.rosetta.core
prp: ModuleType = pyrosetta.rosetta.protocols
pru: ModuleType = pyrosetta.rosetta.utility
prcc: ModuleType = pyrosetta.rosetta.core.conformation
pr_scoring: ModuleType = pyrosetta.rosetta.core.scoring
pr_sele: ModuleType = prc.select.residue_selector

from rdkit import Chem
from rdkit_to_params import Params

def show_cons(pose):
    pi = pose.pdb_info()
    get_atomname = lambda atomid: pose.residue(atomid.rsd()).atom_name(atomid.atomno()).strip()
    get_description = lambda atomid: f'{pose.residue(atomid.rsd()).name3()}{pi.pose2pdb(atomid.rsd()).strip().replace(" ", ":")}.{get_atomname(atomid)}'
    for con in pose.constraint_set().get_all_constraints():
        if con.type() == 'AtomPair':
            print(con.type(), get_description(con.atom1()), get_description(con.atom2()), con.score(pose))
        elif con.type() == 'Angle':
            print(con.type(), get_description(con.atom1()), get_description(con.atom2()), get_description(con.atom3()), con.score(pose))
        else:
            print(con.type(), con.score(pose))

def init_pyrosetta(ignore_unrecognized_res=True, load_PDB_components=False):
    logger = ph.configure_logger()
    pyrosetta.distributed.maybe_init(extra_options=ph.make_option_string(no_optH=False,
                                                                         ex1=None,
                                                                         ex2=None,
                                                                         # mute='all',
                                                                         ignore_unrecognized_res=ignore_unrecognized_res,
                                                                         load_PDB_components=load_PDB_components,
                                                                         ignore_waters=True)
                                     )
    pyrosetta.rosetta.basic.options.set_boolean_option('run:ignore_zero_occupancy', False)
    pyrosetta.rosetta.basic.options.set_boolean_option('in:auto_setup_metals', True)

def constrain_distance(pose, fore_idx, fore_name, aft_idx, aft_name, x0_in=1.334, sd_in=0.2, tol_in=0.02, weight=1):
    AtomPairConstraint = pr_scoring.constraints.AtomPairConstraint  # noqa
    fore = pyrosetta.AtomID(atomno_in=pose.residue(fore_idx).atom_index(fore_name),
                                rsd_in=fore_idx)
    aft = pyrosetta.AtomID(atomno_in=pose.residue(aft_idx).atom_index(aft_name),
                              rsd_in=aft_idx)
    fun = pr_scoring.func.FlatHarmonicFunc(x0_in=x0_in, sd_in=sd_in, tol_in=tol_in)
    if weight != 1:
        fun = pr_scoring.func.ScalarWeightedFunc(weight, fun)
    con = AtomPairConstraint(fore, aft, fun)
    pose.add_constraint(con)
    return con

def constrain_angle(pose, fore_idx, fore_name, mid_name, mid_idx, aft_idx, aft_name, x0_in=109/180*3.14, sd_in=10/180*3.14, weight=1):
    AngleConstraint = pr_scoring.constraints.AngleConstraint
    fore = pyrosetta.AtomID(atomno_in=pose.residue(fore_idx).atom_index(fore_name),
                                rsd_in=fore_idx)
    mid = pyrosetta.AtomID(atomno_in=pose.residue(mid_idx).atom_index(mid_name),
                                rsd_in=mid_idx)
    aft = pyrosetta.AtomID(atomno_in=pose.residue(aft_idx).atom_index(aft_name),
                              rsd_in=aft_idx)
    fun = pr_scoring.func.CircularHarmonicFunc(x0_radians=x0_in, sd_radians=sd_in)
    if weight != 1:
        fun = pr_scoring.func.ScalarWeightedFunc(weight, fun)
    con = AngleConstraint(fore, mid, aft, fun)
    pose.add_constraint(con)
    return con
    
def cartesian_relax(pose, constraint_weight = 5, idxs=(), jumps=(), cycles=3):
    scorefxn: pr_scoring.ScoreFunction = pyrosetta.create_score_function('ref2015_cart')
    scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, constraint_weight)
    scorefxn.set_weight(pr_scoring.ScoreType.angle_constraint, constraint_weight)
    scorefxn.set_weight(pr_scoring.ScoreType.atom_pair_constraint, constraint_weight)
    relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, cycles)
    relax.cartesian(True)
    relax.minimize_bond_angles(True)
    relax.minimize_bond_lengths(True)
    if idxs:
        idx_sele = pr_sele.ResidueIndexSelector()
        for idx in idxs:
            idx_sele.append_index(idx)
        movemap = pyrosetta.MoveMap()
        movemap.set_chi(idx_sele.apply(pose))
        movemap.set_bb(idx_sele.apply(pose))
        for jump in jumps:
            movemap.set_jump(jump, True)
        relax.set_movemap(movemap)
    relax.apply(pose)
    show_cons(pose)
    
def dualspace_relax(pose, constraint_weight = 5, idxs=(), jumps=(), cycles=3):
    scorefxn: pr_scoring.ScoreFunction = pyrosetta.create_score_function('ref2015')
    scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, constraint_weight)
    scorefxn.set_weight(pr_scoring.ScoreType.angle_constraint, constraint_weight)
    scorefxn.set_weight(pr_scoring.ScoreType.atom_pair_constraint, constraint_weight)
    relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, cycles)
    if idxs:
        idx_sele = pr_sele.ResidueIndexSelector()
        for idx in idxs:
            idx_sele.append_index(idx)
        movemap = pyrosetta.MoveMap()
        movemap.set_chi(idx_sele.apply(pose))
        movemap.set_bb(idx_sele.apply(pose))
        for jump in jumps:
            movemap.set_jump(jump, True)
        relax.set_movemap(movemap)
    relax.apply(pose)
    show_cons(pose)
# ------------------- Custom for this project 

def assert_double_xlinked(pose, lig_idx, lig_name):
    ligand = pose.residue(lig_idx)
    assert ligand.name3() == lig_name, ligand.name3()
    assert ligand.n_non_polymeric_residue_connections() == 2, ligand.n_non_polymeric_residue_connections()
    
def constrain_fatA_transition(pose, polymer_chain='A', partner_chain='C', ligand_chain='X'):
    pi = pose.pdb_info()

    ser39 = pi.pdb2pose(res=39, chain=partner_chain)
    pns = pi.pdb2pose(res=1, chain=ligand_chain)
    olx = pi.pdb2pose(res=2, chain=ligand_chain)

    asx262 = pi.pdb2pose(res=262, chain=polymer_chain)
    glh300 = pi.pdb2pose(res=300, chain=polymer_chain)

    hid266=pi.pdb2pose(res=266, chain=polymer_chain)
    asn269=pi.pdb2pose(res=269, chain=polymer_chain)
    asn264=pi.pdb2pose(res=264, chain=polymer_chain)

    # ----------------------------------------------------------------------------------------------------
    # ## Convalents
    # ----------------------------------------------------------------------------------------------------

    # LINK         OG  SEX C  39                 P24 PNS X   1                  1.50
    constrain_distance(pose,
                    fore_idx=ser39,
                    fore_name='OG',
                    aft_idx=pns,
                    aft_name='P24',
                    x0_in=1.50,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=ser39,
                    fore_name='CB',
                    mid_idx=ser39,
                    mid_name='OG',
                    aft_idx=pns,
                    aft_name='P24',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=ser39,
                    fore_name='OG',
                    mid_idx=pns,
                    mid_name='P24',
                    aft_idx=pns,
                    aft_name='O23',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    #LINK         CG  ASP A 262                 O2  OLX X   2                  1.43
    constrain_distance(pose,
                    fore_idx=asx262,
                    fore_name='CG',
                    aft_idx=olx,
                    aft_name='O1',
                    x0_in=1.334,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=asx262,
                    fore_name='OD',
                    mid_idx=asx262,
                    mid_name='CG',
                    aft_idx=olx,
                    aft_name='O1',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=olx,
                    fore_name='C1',
                    mid_idx=olx,
                    mid_name='O1',
                    aft_idx=asx262,
                    aft_name='CG',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    #LINK         S44 PNS X   1                 C1  OLX X   2                  1.81
    constrain_distance(pose,
                    fore_idx=pns,
                    fore_name='S44',
                    aft_idx=olx,
                    aft_name='C1',
                    x0_in=1.82,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=pns,
                    fore_name='C43',
                    mid_idx=pns,
                    mid_name='S44',
                    aft_idx=olx,
                    aft_name='C1',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=pns,
                    fore_name='S44',
                    mid_idx=olx,
                    mid_name='C1',
                    aft_idx=olx,
                    aft_name='C2',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=pns,
                    fore_name='S44',
                    mid_idx=olx,
                    mid_name='C1',
                    aft_idx=olx,
                    aft_name='O1',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=pns,
                    fore_name='S44',
                    mid_idx=olx,
                    mid_name='C1',
                    aft_idx=olx,
                    aft_name='O2',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    # ----------------------------------------------------------------------------------------------------
    # ## hbonding network
    # ----------------------------------------------------------------------------------------------------

    # E[GLH]300 ··· delta H[HID]266
    constrain_distance(pose,
                    fore_idx=glh300,
                    fore_name='OE2',
                    aft_idx=hid266,
                    aft_name='ND1',
                    x0_in=2.7,
                    weight=2
                   )
    # no not flip it
    constrain_distance(pose,
                    fore_idx=glh300,
                    fore_name='OE2',
                    aft_idx=hid266,
                    aft_name='NE2',
                    x0_in=2.7+2.1,
                    weight=2
                   )
    # angle
    constrain_angle(pose,
                    fore_idx=glh300,
                    fore_name='OE2',
                    mid_idx=hid266,
                    mid_name='ND1',
                    aft_idx=hid266,
                    aft_name='NE2',
                    x0_in=3.14,
                   )

    # H[HID]192 epsilon ··· D[ASX]188
    constrain_distance(pose,
                    fore_idx=asx262,
                    fore_name='OD',
                    aft_idx=hid266,
                    aft_name='NE2',
                    x0_in=2.7,
                    weight=2
                   )

    constrain_angle(pose,
                    fore_idx=asx262,
                    fore_name='OD',
                    mid_idx=hid266,
                    mid_name='HE2',
                    aft_idx=hid266,
                    aft_name='NE2',
                    x0_in=3.14,
                    weight=2
                   )

    # N190
    # constrain_distance(pose,
    #                 fore_idx=asn264,
    #                 fore_name='OD1',
    #                 aft_idx=hid266,
    #                 aft_name='H',
    #                 x0_in=1.88
    #                )
    #N195
    constrain_distance(pose,
                    fore_idx=olx,
                    fore_name='O1',
                    aft_idx=asn269,
                    aft_name='N',
                    x0_in=3.10
                   )

def constrain_fatA_substrate(pose, polymer_chain='A', partner_chain='C', ligand_chain='X'):
    pi = pose.pdb_info()

    ser39 = pi.pdb2pose(res=39, chain=partner_chain)
    pns = pi.pdb2pose(res=1, chain=ligand_chain)
    olx = pi.pdb2pose(res=2, chain=ligand_chain)

    asx262 = pi.pdb2pose(res=262, chain=polymer_chain)
    glh300 = pi.pdb2pose(res=300, chain=polymer_chain)

    hid266=pi.pdb2pose(res=266, chain=polymer_chain)
    asn269=pi.pdb2pose(res=269, chain=polymer_chain)
    asn264=pi.pdb2pose(res=264, chain=polymer_chain)
    
    # ----------------------------------------------------------------------------------------------------
    # ## Convalents
    # ----------------------------------------------------------------------------------------------------

    # LINK         OG  SEX C  39                 P24 PNS X   1                  1.50
    constrain_distance(pose,
                    fore_idx=ser39,
                    fore_name='OG',
                    aft_idx=pns,
                    aft_name='P24',
                    x0_in=1.50,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=ser39,
                    fore_name='CB',
                    mid_idx=ser39,
                    mid_name='OG',
                    aft_idx=pns,
                    aft_name='P24',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=ser39,
                    fore_name='OG',
                    mid_idx=pns,
                    mid_name='P24',
                    aft_idx=pns,
                    aft_name='O23',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    #LINK         S44 PNS X   1                 C1  OLX X   2                  1.81
    constrain_distance(pose,
                    fore_idx=pns,
                    fore_name='S44',
                    aft_idx=olx,
                    aft_name='C1',
                    x0_in=1.82,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=pns,
                    fore_name='C43',
                    mid_idx=pns,
                    mid_name='S44',
                    aft_idx=olx,
                    aft_name='C1',
                    x0_in=109/180*3.14,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=pns,
                    fore_name='S44',
                    mid_idx=olx,
                    mid_name='C1',
                    aft_idx=olx,
                    aft_name='C2',
                    x0_in=120/180*3.14,
                    weight=10
                   )

    constrain_angle(pose,
                    fore_idx=pns,
                    fore_name='S44',
                    mid_idx=olx,
                    mid_name='C1',
                    aft_idx=olx,
                    aft_name='O1',
                    x0_in=120/180*3.14,
                    weight=10
                   )

    # ----------------------------------------------------------------------------------------------------
    # ## hbonding network
    # ----------------------------------------------------------------------------------------------------

    # E[GLH]300 ··· delta H[HID]266
    constrain_distance(pose,
                    fore_idx=glh300,
                    fore_name='OE2',
                    aft_idx=hid266,
                    aft_name='ND1',
                    x0_in=2.7,
                    weight=2
                   )
    # no not flip it
    constrain_distance(pose,
                    fore_idx=glh300,
                    fore_name='OE2',
                    aft_idx=hid266,
                    aft_name='NE2',
                    x0_in=2.7+2.1,
                    weight=2
                   )
    # angle
    constrain_angle(pose,
                    fore_idx=glh300,
                    fore_name='OE2',
                    mid_idx=hid266,
                    mid_name='ND1',
                    aft_idx=hid266,
                    aft_name='NE2',
                    x0_in=3.14,
                   )

    # H[HID]192 epsilon ··· D[ASX]188
    constrain_distance(pose,
                    fore_idx=asx262,
                    fore_name='OD1',
                    aft_idx=hid266,
                    aft_name='NE2',
                    x0_in=2.7,
                    weight=2
                   )

    constrain_angle(pose,
                    fore_idx=asx262,
                    fore_name='OD1',
                    mid_idx=hid266,
                    mid_name='HE2',
                    aft_idx=hid266,
                    aft_name='NE2',
                    x0_in=3.14,
                    weight=2
                   )

    # N190
    # constrain_distance(pose,
    #                 fore_idx=asn264,
    #                 fore_name='OD1',
    #                 aft_idx=hid266,
    #                 aft_name='H',
    #                 x0_in=1.88
    #                )
    #N195
    constrain_distance(pose,
                    fore_idx=olx,
                    fore_name='O1',
                    aft_idx=asn269,
                    aft_name='N',
                    x0_in=3.10
                   )
# ---------------------------
init_pyrosetta(True)

## Disulfide fix

In [ ]:
original = pyrosetta.pose_from_file('x1747-mashed.min.pdb')

pdb2pose = original.pdb_info().pdb2pose
prcc.form_disulfide(original.conformation(), pdb2pose(res=166, chain='A'), pdb2pose(res=221, chain='A'))
prcc.form_disulfide(original.conformation(), pdb2pose(res=166, chain='B'), pdb2pose(res=221, chain='B'))
# this was actually not needed
constrain_distance(original, pdb2pose(res=166, chain='A'), 'SG', pdb2pose(res=221, chain='A'), 'SG', x0_in=2.05)
constrain_distance(original, pdb2pose(res=166, chain='A'), 'SG', pdb2pose(res=221, chain='A'), 'SG', x0_in=2.05)

show_cons(original)

pose = original.clone()
dualspace_relax(pose, idxs=pdb2pose(res=166, chain='A'), 'SG', pdb2pose(res=221, chain='A'))
pose.dump_pdb('x1747-mashed.min.pdb')

## Ligand switch

Whereas Fragmenstein can do single linked, double linked is bound to get nasty so was done manually.

merger.pdb is a textedit mix of 1R3 and protein and LINK lines.

LINK         NE2 HIS A 192                 C51 1R3 X   1     1555   1555  1.30
LINK         OG  SER C  39                 P02 1R3 X   1     1555   1555  1.60

In [ ]:
from rdkit.Chem import Draw

changed = {'present': 'CCCCCCCCCS(CCCNC(=O)CCNC(=O)[C@@H](C(C)(C)COP(=O)(*)-[O-])[OH])(O*)-[O-]',
            'desired': 'CCCCCCCCCC(SCCNC(=O)CCNC(=O)[C@@H](C(C)(C)COP(=O)(*)-[O-])[OH])(O*)-[O-]'}

fig = Draw.MolsToGridImage(list(map(Chem.MolFromSmiles, changed.values())), legends=list(changed.keys()))
with open("ligand_change.png", "wb") as png:
    png.write(fig.data)
fig

## Parameterise

In [ ]:
from rdkit import Chem

# mod
smiles = 'CCCCCCCCCS(CCCNC(=O)CCNC(=O)[C@@H](C(C)(C)COP(=O)(*)-[O-])[OH])(O*)-[O-]'
block = '\n'.join([l for l in Path('merger.pdb').read_text().split('\n') if 'HETATM' in l])
params = Params.from_smiles_w_pdbblock(pdb_block=block, smiles=smiles, name='LIG')
params.dump('1R3.params')
Chem.MolFromSmiles(smiles)

## Load Merger

In [ ]:
pose: pyrosetta.Pose = ph.pose_from_file('merger.pdb', ['1R3.params'])
# it's the last residue
assert_double_xlinked(pose, lig_idx=pose.total_residue(), lig_name=params.NAME)
# repeated below anyway...
# AF2 was never corrected for an +74 offset
...
cartesian_relax(...)
dualspace_relax(...)
pose.dump_pdb('final_first.pdb')

## Second ligand switch

In [ ]:
## Get mapping

from rdkit import Chem
from rdkit.Chem import AllChem, rdFMCS

# Load PDB files
fat_pdb = Chem.MolFromPDBFile('mergerV2.pdb')
fat_residue = Chem.SplitMolByPDBChainId(fat_pdb)['X']
pns_residue = Chem.MolFromPDBFile('PDB-PNS.pdb')
ola_residue = Chem.MolFromPDBFile('PDB-OLA.pdb')

def get_mcs_mapping(key_mol, val_mol):
    mcs = rdFMCS.FindMCS([key_mol, val_mol])
    mcs_pattern = Chem.MolFromSmarts(mcs.smartsString)
    key_match = key_mol.GetSubstructMatch(mcs_pattern)
    val_match = val_mol.GetSubstructMatch(mcs_pattern)
    assert key_match
    assert val_match
    return dict(zip(key_match, val_match))

get_name = lambda res, idx: res.GetAtomWithIdx(idx).GetPDBResidueInfo().GetName()
fat2pns = {get_name(fat_residue, fi): get_name(pns_residue, pi) for fi, pi in get_mcs_mapping(fat_residue, pns_residue).items()}
fat2ola = {get_name(fat_residue, fi): get_name(ola_residue, pi) for fi, pi in get_mcs_mapping(fat_residue, ola_residue).items()}

In [ ]:
## Make file

block = ''
for line in Path('mergerV2.pdb').read_text().split('\n'):
    if 'HETATM' not in line or 'FAT' not in line:
        block += line + '\n'
        continue
    name = line[12:16]
    if name in fat2pns: 
        newname = fat2pns[name]
        newres = 'PNS'
        newidx = '1'
        block += line[:12] + f'{newname} {newres} X   {newidx}' + line[26:] + '\n'
    elif name in fat2ola: 
        newname = fat2ola[name]
        newres = 'OLX' # it's not quite OLA
        newidx = '2'
        block += line[:12] + f'{newname} {newres} X   {newidx}' + line[26:] + '\n'
    elif name.strip()[0] == 'H':
        continue
    else:
        raise Exception(line)
        
Path('mergerV2.1.pdb').write_text(block)

In [ ]:
## Parameterise

from rdkit_to_params import Params
from pathlib import Path
from rdkit import Chem

pdb_path = Path('mergerV2.2.pdb')

# mod from https://www.rcsb.org/ligand/OLA or https://www.rcsb.org/ligand/ELA
smiles = 'CCCCCCCC\C=C/CCCCCCCC(-[O-])(O*)*'
block = '\n'.join([l for l in pdb_path.read_text().split('\n') if 'HETATM' in l])
params = Params.from_smiles_w_pdbblock(pdb_block=block, smiles=smiles, name='OLX')
params.dump('OLX.params')
display(Chem.MolFromSmiles(smiles))

smiles = 'CC(C)(COP(=O)([O-])*)[C@H](C(=O)NCCC(=O)NCCS*)O'
block = '\n'.join([l for l in pdb_path.read_text().split('\n') if 'HETATM' in l])
params = Params.from_smiles_w_pdbblock(pdb_block=block, smiles=smiles, name='PNS')
params.dump('PNS.params')
display(Chem.MolFromSmiles(smiles))

## Load
The file was changes appropriately

In [ ]:
# mergerV2.2.pdb
pose: pyrosetta.Pose = ph.pose_from_file('wiggled.pdb', ['SEX.params', 'OLX.params', 'PNS.params', 'GLH.params'])
pi = pose.pdb_info()

# HIE266
#prp.simple_moves.MutateResidue(target=pi.pdb2pose(res=266, chain='A'), new_res='HIS_E').apply(pose)

# GLU300  nucleophile
glh300 = pi.pdb2pose(res=300, chain='A')
if pose.residue(glh300).name3() == 'GLU':
    prp.simple_moves.MutateResidue(target=glh300, new_res='GLH').apply(pose)

In [ ]:
assert_double_xlinked(pose, lig_idx=pose.total_residue(), lig_name='OLX')
assert_double_xlinked(pose, lig_idx=pose.total_residue() - 1, lig_name='PNS')

## Minimisation
First cartesian of relevant residues, the globally

In [ ]:
ser39 = pi.pdb2pose(res=39, chain=partner_chain)
pns = pi.pdb2pose(res=1, chain=ligand_chain)
olx = pi.pdb2pose(res=2, chain=ligand_chain)

asx262 = pi.pdb2pose(res=262, chain=polymer_chain)
glh300 = pi.pdb2pose(res=300, chain=polymer_chain)

hid266=pi.pdb2pose(res=266, chain=polymer_chain)
asn269=pi.pdb2pose(res=269, chain=polymer_chain)
asn264=pi.pdb2pose(res=264, chain=polymer_chain)
# move c**kblocking arg298, 299
arg298=pi.pdb2pose(res=298, chain='A')
arg299=pi.pdb2pose(res=299, chain='A')
neigh261=pi.pdb2pose(res=261, chain='A')
neigh263=pi.pdb2pose(res=261, chain='A')
neigh301=pi.pdb2pose(res=261, chain='A')

cartesian_relax(pose,
                cycles=15,
                constraint_weight=1,
                idxs=[ser39, pns, olx, neigh261, asx262,neigh263, asn269,asn264,hid266,  arg298, arg299, glh300, neigh301], 
                jumps=[pose.num_jump() -1, pose.num_jump()])
pose.dump_pdb('cartA.pdb')

cartesian_relax(pose,
                cycles=3,
                constraint_weight=0.5,
                idxs=[ser39, pns, olx, neigh261, asx262,neigh263, asn269,asn264,hid266,  arg298, arg299, glh300, neigh301], 
                jumps=[pose.num_jump() -1, pose.num_jump()])
pose.dump_pdb('cartB.pdb')

cartesian_relax(pose,
                cycles=3,
                constraint_weight=1,
                idxs=[ser39, pns, olx, neigh261, asx262,neigh263, asn269,asn264,hid266,  arg298, arg299, glh300, neigh301], 
                jumps=[pose.num_jump() -1, pose.num_jump()])
pose.dump_pdb('cartC.pdb')

dualspace_relax(pose, idxs=[ser39,pns, olx, asx262, glh300, hid266, asn269,asn264, arg298, arg299], jumps=[pose.num_jump() -1, pose.num_jump()])
pose.dump_pdb('regD.pdb')

dualspace_relax(pose, 0)
pose.dump_pdb('final.pdb')

## Product form

In [ ]:
## Parameterise

from rdkit_to_params import Params
from pathlib import Path
from rdkit import Chem

pdb_path = Path('mergerV2.2.pdb')

# mod from https://www.rcsb.org/ligand/OLA or https://www.rcsb.org/ligand/ELA
smiles = 'CCCCCCCC\C=C/CCCCCCCC(=O)*'
block = '\n'.join([l for l in pdb_path.read_text().replace('OLX', 'OLA').split('\n') if 'HETATM' in l and 'O2' not in l])
params = Params.from_smiles_w_pdbblock(pdb_block=block, smiles=smiles, name='OLA')
params.dump('OLA.params')
display(Chem.MolFromSmiles(smiles))

In [ ]:
## Load

...